# Sayou Chunking Quick Start

**Sayou Fabric**의 텍스트 분할 엔진인 `sayou-chunking`을 시연합니다.
Refinery에서 정제된 데이터를 받아, RAG(검색 증강 생성)에 최적화된 크기와 문맥을 가진 **Chunk**로 자릅니다.

**핵심 기능:**
1. **Recursive Split:** 문맥(문단, 문장)을 유지하며 안전하게 자르기
2. **Markdown Aware:** 헤더(#) 구조를 인식하여 의미 단위로 자르기
3. **Semantic Split:** 문장의 의미적 유사도를 분석하여 주제별로 자르기
4. **Plugin System:** 사용자 정의 분할기(Audited 등) 확장

In [ ]:
import logging
import json
from sayou.chunking.pipeline import ChunkingPipeline

# 플러그인 임포트
from sayou.chunking.plugins.markdown_splitter import MarkdownSplitter
from sayou.chunking.plugins.audited_fixed_length_splitter import AuditedFixedLengthSplitter

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Import Successful!")

## 1. 파이프라인 초기화 (Initialization)

파이프라인을 생성하고 플러그인을 등록합니다.

In [ ]:
# 커스텀 플러그인 인스턴스 생성
markdown_plugin = MarkdownSplitter()
audited_plugin = AuditedFixedLengthSplitter()

# 파이프라인 생성 시 extra_splitters로 주입
pipeline = ChunkingPipeline(extra_splitters=[markdown_plugin, audited_plugin])

# 초기화 (전역 설정 주입 가능)
pipeline.initialize()

## 2. 시나리오 1: Markdown 구조적 분할

가장 강력한 기능 중 하나입니다. Markdown 헤더 구조를 인식하여 청킹합니다.
* **입력:** 헤더, 리스트, 코드가 포함된 마크다운 텍스트
* **전략:** `strategy="markdown"`

In [ ]:
md_content = """
# Sayou Chunking Guide

Chunking is essential for RAG.

## Strategy 1: Recursive
It splits text by separators like newlines.

## Strategy 2: Markdown
It respects the document structure.
- Preserves Headers
- Protects Code Blocks

| Strategy | Context |
| :--- | :--- |
| Fixed | Low |
| Semantic | High |
""".strip()

request_md = {
    "content": md_content,
    "metadata": {"source": "guide.md"},
    "config": {"chunk_size": 100}
}

print(">>> Running Markdown Splitter...")
chunks = pipeline.run(request_md, strategy="markdown")

for i, chunk in enumerate(chunks):
    # Semantic Type과 Header 여부 확인
    sem_type = chunk.metadata.get("semantic_type", "text")
    is_header = chunk.metadata.get("is_header", False)
    print(f"[{i}] [{sem_type}] {chunk.content[:50]}...")

## 3. 시나리오 2: Recursive (기본) 분할

일반 텍스트를 문단 -> 문장 순서로 안전하게 자릅니다.
* **전략:** `strategy="recursive"` (또는 default)

In [ ]:
plain_text = "This is paragraph one.\n\nThis is paragraph two. It has multiple sentences. We want to keep them together if possible.\n\nParagraph three."

request_rec = {
    "content": plain_text,
    "metadata": {"source": "plain.txt"},
    "config": {"chunk_size": 50, "chunk_overlap": 10}
}

print(">>> Running Recursive Splitter...")
chunks_rec = pipeline.run(request_rec, strategy="recursive")

for i, chunk in enumerate(chunks_rec):
    print(f"[{i}] Len: {len(chunk.content)} | {chunk.content}")

## 4. 시나리오 3: 커스텀 플러그인 (Audited)

우리가 추가한 `AuditedFixedLengthSplitter`를 사용해 봅니다.
청크 메타데이터에 처리 시간과 원본 길이 정보(Audit)가 남아야 합니다.
* **전략:** `strategy="audited_fixed"`

In [ ]:
request_audit = {
    "content": "A" * 200, # 200자 더미 텍스트
    "metadata": {"source": "audit_log.txt"},
    "config": {"chunk_size": 50, "chunk_overlap": 0}
}

print(">>> Running Audited Fixed Splitter...")
chunks_audit = pipeline.run(request_audit, strategy="audited_fixed")

for i, chunk in enumerate(chunks_audit):
    # 메타데이터에 'audit' 필드가 있는지 확인
    audit_info = chunk.metadata.get("audit", {})
    print(f"[{i}] Content: {chunk.content}")
    print(f"    Audit: {audit_info}")

## 5. 결과 저장 (JSON Serialization)

생성된 `Chunk` 객체는 Pydantic 모델이므로, `model_dump()`를 통해 딕셔너리로 변환하여 저장합니다.

In [ ]:
# 객체 리스트 -> 딕셔너리 리스트 변환
output_data = [c.model_dump() for c in chunks_audit]

with open("chunks_output.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)
    
print(f"\n✅ Saved {len(output_data)} chunks to chunks_output.json")

In [ ]:
# Cleanup
import os
if os.path.exists("chunks_output.json"):
    os.remove("chunks_output.json")
print("Cleanup complete.")